In [ ]:
%%html

<style>    
    @import url("css/custom_styles.css")
</style>

<center>
    <h1>
    Transformation Et Manipulation Des Données<br>
    </h1>
    MovieLens - Système de recommendations de films par regroupement<br>
    <br>
    <b>Jean-Francois Gagnon</b><br>
    <br>
    420-A56<br>
    <br>
</center>

# Introduction

<font class="answer">
    
J'ai choisi la base de données [MovieLens Small](https://tinyurl.com/bdhmcfht). Elle se compose de 100836 notes représentant 9742 films. Elle contient également les métadonnées sur 19 genres et des liens sur [TMDB](https://www.themoviedb.org) qui permettront d'augmemter son contenu.
    
Plus spécifiquement, (Voir https://files.grouplens.org/datasets/movielens/ml-latest-small-README.html et décrire en détails le format)
    
    
    
L'objectif de ce projet est de contruire un système de recommentation de films en utilisant les techniques de regroupement vu dans le cours. Les détails du sytème seront abordés plus loins dans ce notebooks

In [ ]:
#
# imports utilitaires
#

%matplotlib inline

import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import requests
import seaborn as sns
import time

from bs4 import BeautifulSoup
from imblearn.under_sampling import RandomUnderSampler
from sklearn.cluster import MiniBatchKMeans
from sklearn.decomposition import TruncatedSVD

from tqdm.notebook import tqdm

In [ ]:
#
# imports faisant partie de nos propres modules
#

import helpers as hlp
import helpers.dataset.MovieLens as mvl

from helpers.jupyter import display_html

#
# faudrait mettre nos utilitaires de clusterings ici
#

#
# fonctions utilitaires
#
def show_na_simple(dataframe):
    na_ = dataframe.isna().sum().to_frame()
    na_.columns = ["NA"]
    display(na_.T)

def show_types_simple(dataframe):
    types_ = dataframe.dtypes.to_frame()
    types_.columns = ["Type"]
    display(types_.T)
    
def split_title_year(title, regex):
    x = regex.search(title)
    if x:
        title = x.group(1)
        year = x.group(2)
        year = int(year) if year else pd.NA
    else:
        year = pd.NA
    
    return pd.Series({"title": title, "year": year})

def web_scapping_imdb(imdbId_series, config, callback):
    count = links_series.shape[0]
    for index, id in tqdm(imdbId_series.items(), total=count):
        url = "".join([config.scrapping_imdb_url, str(id)])
        with requests.get(url, headers=config.scrapping_imdb_headers.__dict__) as r:
            callback(r, index, id)
    
def json_nested_object(json, keys_iterable):
    for k in keys_iterable:
        if not k in json:
            return None
        json = json[k]
        
    return json

# Prétraitement

<font class="answer">

Description ici?

In [ ]:
#
# parametres configurant nos traitemens
#
configs = hlp.get_configs("config_overrides.json")

#
# obtenir le dataset
#
mvl_dataset = mvl.load(configs.dataset)

## links.csv

<font class="answer">
    
links.csv n'est pas utilisé directement pour le clustering. Cependant, il le sera pour complémenter l'information des autres base de données. Il m'apparait donc imporant de faire un survol rapide.

In [ ]:
print("Links")

show_na_simple(mvl_dataset.links)
show_types_simple(mvl_dataset.links)
display(mvl_dataset.links.head())

<font class="answer">
    
Il manque quelques liens sur [TMDB](https://www.themoviedb.org/) (ce qui explique le type float64). Les liens [IMDB](https://www.imdb.com/) seront par conséquent privélégiés pour fin de web scrapping.

## movies.csv

<font class="answer">
    
Mettre description ici?

In [ ]:
print("Movies")

show_na_simple(mvl_dataset.movies)
show_types_simple(mvl_dataset.movies)
display(mvl_dataset.movies.head())

<font class="answer">

Tel que décris par MovieLens, *title* contient l'année de parution. Nous allons l'extraire afin d'obtenir une nouvelle variable. *title* sera adressé un pleu plus bas.

In [ ]:
title_year_re = re.compile(configs.dataset.title_regex, flags=0)        
title_year = mvl_dataset.movies.title.apply(split_title_year, args=(title_year_re,))

# validation de l'extraction
print("Informations extraites")

show_na_simple(title_year)
show_types_simple(title_year)
display(title_year.head())

<font class="answer">
   
On peut remarquer qu'il manque quelques années de parution. Il est probablement possible de les fixer en utilisant *links.imdbId* avec le web scrapping.

In [ ]:
def imdb_scap_year(response, index, id):
    success = False
    if response.ok:
        bs = BeautifulSoup(response.text, "html.parser")

        imdb_info = bs.find("script", attrs={"type": "application/ld+json"})
        imdb_json = json.loads(imdb_info.text)
        date_published = json_nested_object(imdb_json, ["datePublished"])
        if not success and date_published:
            date_published = pd.to_datetime(date_published)
            year = date_published.year
            success = True

        imdb_info = bs.find("script", attrs={"type": "application/json"})
        imdb_json = json.loads(imdb_info.text)
        releaseYear = json_nested_object(imdb_json, 
                                         ["props", 
                                          "pageProps", 
                                          "mainColumnData", 
                                          "releaseYear",
                                          "year"])
        if not success and releaseYear:
            year = releaseYear
            success = True

    if success:
        title_year.year[index] = year
    else:
        print(url, title_year.title[index], "Failed")

# corriger year via web scrapping avec imdb
year_na = title_year.year.isna()
year_link = mvl_dataset.links.imdbId[ year_na ]

if year_link.shape[0] > 0:
    web_scapping_imdb(year_link, configs.dataset, imdb_scap_year)

# validation du scapping
show_na_simple(title_year)

In [ ]:
# mettre a jour movies
mvl_dataset.movies["year"] = title_year.year.astype(np.int64)

In [ ]:
#
# validation effet extraction de l'annee de title
#
imdb_ids = mvl_dataset.links.imdbId[mvl_dataset.links.movieId == mvl_dataset.movies.movieId]

title = mvl_dataset.movies[["movieId", "year"]].copy()
title["title"] = title_year.title.copy()
title["imdbId"] = imdb_ids.copy()

def title_agregate(dataframe):
    return pd.Series({"imdbIds": dataframe.imdbId.unique(),
                      "years": dataframe.year.unique(),
                      "counts": dataframe.movieId.nunique()})

groups = title.groupby("title").apply(title_agregate)
groups.sort_values(by="counts", ascending=False, inplace=True)

display(groups.head())

<font class="answer">
    
Une inspection manuelle de [Hamlet 1996](http://www.imdb.com/title/tt0116477) et [Hamlet 1948](http://www.imdb.com/title/tt0040416) via IMDB permet de voir que l'année juxtaposée au titre est un identifiant unique. Je dois donc garder *title* original.

In [ ]:
# validation doublons
print("Avant:", mvl_dataset.movies.shape)
mvl_dataset.movies.drop_duplicates(inplace=True)
print("Après:", mvl_dataset.movies.shape)

<font class="answer">
Aucun doublon

In [ ]:
# genre est de type categoriel
# changer string en list en prevision de pandas.get_dummies() 
def genres_str_to_array(genres, splitter):
    if isinstance(genres, str):
        return genres.split(splitter)
    else:
        return genres

mvl_dataset.movies.genres = mvl_dataset.movies.genres.apply(genres_str_to_array, 
                                                            args=(configs.dataset.genre_splitter,)).copy()

# validation changement de type
display(mvl_dataset.movies.head())

In [ ]:
# validation modalites genres
def gather_genres(genres, final_set):
    final_set.update(genres)
    
def show_unique_genres():
    genres_set = set()
    mvl_dataset.movies.genres.apply(gather_genres,
                                    args=(genres_set,))
    
    print("Genres:", len(genres_set), "modalités")
    display(genres_set)
    
show_unique_genres()

<font class="answer">

Étonnament, IMAX est un genre. 

In [ ]:
# valider gernes IMAX
def find_in_list(haystack, needle):
    return needle in haystack

def list_size(list_):
    return len(list_)

imax = mvl_dataset.movies[ mvl_dataset \
                              .movies \
                              .genres \
                              .apply(find_in_list, needle="IMAX") ].copy()
imax["counts"] = imax.genres.apply(list_size)
print("IMAX", imax.shape[0], "- min genres count", imax.counts.min())
display(imax.head())

<font class="answer">
IMAX est en fait utilisé comme un "attribut"; il n'est jamais utlisé seul.

In [ ]:
def show_no_genres():
    no_genres = mvl_dataset.movies[ mvl_dataset
                                       .movies \
                                       .genres \
                                       .apply(find_in_list, needle="(no genres listed)") ]
    print("(no genres listed)", no_genres.shape[0])
    display(no_genres.head())
    
show_no_genres()

<font class="answer">
(no genres listed) représente des valeurs manquantes. Je vais utiliser le web scrapping pour tenter d'y remédier.

In [ ]:
def imdb_scap_genres(response, index, id):
    success = False
    if response.ok:
        bs = BeautifulSoup(response.text, "html.parser")

        imdb_info = bs.find("script", attrs={"type": "application/ld+json"})
        imdb_json = json.loads(imdb_info.text)
                
        genres = json_nested_object(imdb_json, ["genre"])
        if not success and genres:
            success = True

#         imdb_info = bs.find("script", attrs={"type": "application/json"})
#         imdb_json = json.loads(imdb_info.text)
#         releaseYear = json_nested_object(imdb_json, 
#                                          ["props", 
#                                           "pageProps", 
#                                           "mainColumnData", 
#                                           "releaseYear",
#                                           "year"])
#         if not success and releaseYear:
#             year = releaseYear
#             success = True

    if success:
        mvl_dataset.movies.genres[index] = genres
    else:
        print()
        print( json.dumps(imdb_json, indent=4) )
        print()
        
        print(response.url, mvl_dataset.movies.title[index], "Failed")

# corriger genres via web scrapping avec imdb
genres_imdbIds = mvl_dataset.links.imdbId[no_genres.index]

if genres_imdbIds.shape[0] > 0:
    web_scapping_imdb(genres_imdbIds, configs.dataset, imdb_scap_genres)

show_unique_genres()
show_no_genres()

## ratings.csv

<font class="answer">
    
Mettre description ici?

In [ ]:
print("Ratings")

show_na_simple(mvl_dataset.ratings)
show_types_simple(mvl_dataset.ratings)
display(mvl_dataset.ratings.head())

In [ ]:
# explorer

# Exploration